In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Download the drive first:

https://drive.google.com/drive/folders/1gvTs0901Xc1yAgj8_ugqGOuZ_d190Ikg

In [2]:
%run "/content/drive/MyDrive/CRHM AIST2010/Harmonization_Testground.ipynb"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 22.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592284 sha256=12b5040e7c152bf112cd67115abcc48f18a0f6fe96f056d60ebb2f1a7d0680e6
  Stored in directory: /root/.cache/pip/wheels/cd/a5/30/7b8b7f58709f5150f67f98fde4b891ebf0be9ef07a8af49f25
Successfully built pretty_midi
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.7/90.7 kB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 18.9 MB/s eta 0:00:00
  Created wheel for mir-eval: filename=mir_eval-0.7-py3-none-any.whl size=100703 sha256=2acb4c891ab290cfbd1a963834627615293d32ccbae252fcb60da1e0bb58bf43
  Stored in directory: /root/.cache/pip/wheels/3e/2f/0d/dda9c4c

In [21]:
%run "/content/drive/MyDrive/CRHM AIST2010/Chord_Recognition_Testground.ipynb"

In [4]:
%run "/content/drive/MyDrive/CRHM AIST2010/LSTM_testground.ipynb"

In [5]:
%run "/content/drive/MyDrive/CRHM AIST2010/Vocoder.ipynb"

Upload your audio file

Please select the key and tempo of your audio as well

In [6]:
from google.colab import files
import ipywidgets as widgets

# Prompt the user to upload a file
print("Please upload vocal file: ")
uploaded = files.upload()

# Access the uploaded file
for filename, content in uploaded.items():
    # Save the uploaded file
    vfp = "/content/" + filename
    print(vfp)

print("Please upload accompaniment file (optional): ")
uploaded = files.upload()

# Access the uploaded file
if uploaded == {}:
  afp = None
else:
  for filename, content in uploaded.items():
      # Save the uploaded file
      afp = "/content/" + filename
      print(afp)

# Select the key
key_input = widgets.Dropdown(
    options= ['C', 'C#', 'D', "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B", "Cm", "C#m", "Dm", "D#m", "Em", "Fm", "F#m", "Gm", "G#m", "Am", "A#m", "Bm", "None"],
    value='C',
    description='Key:',
    disabled=False,
)
display(key_input)





Please upload vocal file: 


Saving viva-la-vida-official-video.wav to viva-la-vida-official-video.wav
/content/viva-la-vida-official-video.wav
Please upload accompaniment file (optional): 


Saving viva-la-vida-official-video.wav to viva-la-vida-official-video (1).wav
/content/viva-la-vida-official-video (1).wav


Dropdown(description='Key:', options=('C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B', 'Cm', '…

Method 1: Template matching

In [20]:
if afp != None:
  chord_pred, sr, abc, abcd = template_predict(afp, key_input.value, key_filtering = True, version = "STFT")
  print([chord_map[tuple_item[0]] for tuple_item in chord_pred])
  chord_pred = mode_filter(chord_pred)
  chord_pred = mode_filter(chord_pred)
  chord_pred = mode_filter(chord_pred)
  print([chord_map[tuple_item[0]] for tuple_item in chord_pred])
else:
  chord_pred, sr, abc, abcd = template_predict(vfp, key_input.value, tempo_input.value, key_filtering = True)


TypingError: ignored

Method 2: Bi-directional Long Short-Term Memory (Bi-LSTM)

In [ ]:
# audio processing
if afp != None:
  X = preprocess_librosa(afp)
else:
  X = preprocess_librosa(vfp)

y_size, y_ind = 25, -8 # label size = 25,
model = LSTMClassifier(input_size=84, hidden_dim=200, output_size=y_size,
                        num_layers=2,
                        use_gpu=torch.cuda.is_available(), bidirectional=True, dropout=(0.4, 0.0, 0.0))

# load the pretrained model
if torch.cuda.is_available():
    model = model.cuda()
    model.load_state_dict(torch.load("/content/drive/MyDrive/CRHM AIST2010/LSTM1.1_opt_SGD"))
else:
    model.load_state_dict(torch.load("/content/drive/MyDrive/CRHM AIST2010/LSTM1.1_opt_SGD", map_location='cpu'))
model.eval()
result = t(model, X)

lstm_chord_map = {
    "C:maj": 0,
    "C#:maj": 1,
    "D:maj": 2,
    "D#:maj": 3,
    "E:maj": 4,
    "F:maj": 5,
    "F#:maj": 6,
    "G:maj": 7,
    "G#:maj": 8,
    "A:maj": 9,
    "A#:maj": 10,
    "B:maj": 11,
    "C:min": 12,
    "C#:min": 13,
    "D:min": 14,
    "D#:min": 15,
    "E:min": 16,
    "F:min": 17,
    "F#:min": 18,
    "G:min": 19,
    "G#:min": 20,
    "A:min": 21,
    "A#:min": 22,
    "B:min": 23,
    "N": 24,
}
result = [(lstm_chord_map[c], s, e) for s, e, c in result]
lstm_pred = mode_filter(result)
print(lstm_pred)

[(4, 0.0, 0.13931972789115646), (4, 0.13931972789115646, 0.7430385487528345), (4, 0.7430385487528345, 1.3003174603174603), (4, 1.3003174603174603, 4.08671201814059), (4, 4.08671201814059, 4.2260317460317465), (4, 4.2260317460317465, 4.876190476190477), (4, 4.876190476190477, 6.548027210884354), (4, 6.548027210884354, 7.244625850340136), (4, 7.244625850340136, 7.476825396825397), (4, 7.476825396825397, 8.219863945578231), (9, 8.219863945578231, 8.684263038548753), (11, 8.684263038548753, 9.148662131519275), (9, 9.148662131519275, 9.427301587301587), (11, 9.427301587301587, 10.356099773242631), (11, 10.356099773242631, 11.795736961451247), (9, 11.795736961451247, 13.096054421768708), (11, 13.096054421768708, 13.188934240362812), (9, 13.188934240362812, 14.489251700680272), (9, 14.489251700680272, 14.814331065759637), (9, 14.814331065759637, 17.60072562358277), (9, 17.60072562358277, 19.365442176870747), (9, 19.365442176870747, 20.433560090702947), (9, 20.433560090702947, 20.5728798185941

Chord Symbol Recall

CSR = $\frac{\text{total duration of segments where prediction is correct}}{\text{total duration of the song}}$ x 100%

We will use the above metric to evaluate the accuracy of both methods

In [ ]:
def accuracy_evaluation(X, ground_truth):



The output of the chord prediction methods has the following output format:

[$t_1$, $t_2$, $chord$] for every time interval $\in$ [$t_1$, $t_2$]

Then, we feed the prediction into our harmonization

In [ ]:
lstm = True
if lstm != True:
  soprano, alto, tenor, bass = harmonization(vfp, chord_pred, "sample_t.midi")
else:
  soprano, alto, tenor, bass = harmonization(vfp, lstm_pred, "sample_l.midi")



Predicting MIDI for /content/ji-yi-mian-pillow-talk-official-music-video [vocals].wav...


The harmonization gives a four-wayutput

[$pitch$, $t_1$, $t_2$] for every time interval $in$ [$t_1$, $t_2$]

which is then inputted into the vocoder

In [ ]:
!pip install pydub

In [ ]:
from pydub import AudioSegment

gen_voc(vfp, "sample_l_s.wav", soprano)
gen_voc(vfp, "sample_l_a.wav", alto)
gen_voc(vfp, "sample_l_t.wav", tenor)
gen_voc(vfp, "sample_l_b.wav", bass)
# Load the audio tracks
track1 = AudioSegment.from_file("sample_l_s.wav")
track2 = AudioSegment.from_file("sample_l_a.wav")
track3 = AudioSegment.from_file("sample_l_t.wav")
track4 = AudioSegment.from_file("sample_l_b.wav")

track1 = track1  # Decrease volume by 10 dB
track2 = track2 - 1  # Increase volume by 3 dB
track3 = track3 - 1 # Apply a fade-in and fade-out effect
track4 = track4 - 1 # Increase volume by 6 dB

# Mix the tracks
output = track1.overlay(track2).overlay(track3).overlay(track4)

# Export the mixed track to a file
output.export("mixed_sample_ls.wav", format="wav")

Done
Done
Done
Done


<_io.BufferedRandom name='mixed_sample_ls.wav'>

In [ ]:
from pydub import AudioSegment

gen_voc(vfp, "sample_t_s.wav", soprano)
gen_voc(vfp, "sample_t_a.wav", alto)
gen_voc(vfp, "sample_t_t.wav", tenor)
gen_voc(vfp, "sample_t_b.wav", bass)
# Load the audio tracks
track1 = AudioSegment.from_file("sample_t_s.wav")
track2 = AudioSegment.from_file("sample_t_a.wav")
track3 = AudioSegment.from_file("sample_t_t.wav")
track4 = AudioSegment.from_file("sample_t_b.wav")

track1 = track1  # Decrease volume by 10 dB
track2 = track2 - 2   # Increase volume by 3 dB
track3 = track3 - 2  # Apply a fade-in and fade-out effect
track4 = track4 - 2 # Increase volume by 6 dB

# Mix the tracks
output = track1.overlay(track2).overlay(track3).overlay(track4)

# Export the mixed track to a file
output.export("mixed_sample_t.wav", format="wav")

Done
Done
Done
Done


<_io.BufferedRandom name='mixed_sample_l.wav'>